In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [2]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10, 20):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t2 = time.time()
print(f'before flush {(t2 - t0):.2f} seconds')
producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 10}
Sent: {'number': 11}
Sent: {'number': 12}
Sent: {'number': 13}
Sent: {'number': 14}
Sent: {'number': 15}
Sent: {'number': 16}
Sent: {'number': 17}
Sent: {'number': 18}
Sent: {'number': 19}
before flush 0.55 seconds
took 0.55 seconds


In [3]:
import pandas as pd

In [4]:
df_green = pd.read_csv('./data/green_tripdata_2019-10.csv.gz', low_memory=False)

columns = {
    'lpep_pickup_datetime': 'pickup_datetime', 
    'lpep_dropoff_datetime': 'dropoff_datetime', 
    'PULocationID': 'pu_location_id', 
    'DOLocationID': 'do_location_id', 
    'passenger_count': 'passenger_count', 
    'trip_distance': 'trip_distance', 
    'tip_amount': 'tip_amount'
}
df_green = df_green[columns.keys()].rename(columns=columns)

topic_name = 'green-trips-2'

t0 = time.time()

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
    # print(f"Sent: {row_dict}")
    # break

    #TODO implement sending the data here

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 23.07 seconds


In [5]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/21 02:38:31 WARN Utils: Your hostname, Andreys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.10.123 instead (on interface en0)
24/03/21 02:38:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/andrey_groza/.ivy2/cache
The jars for the packages stored in: /Users/andrey_groza/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d7666ebd-3372-4b8c-9995-c9fcd1c1edaf;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 291ms :: artifacts dl 5ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

In [6]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips-2") \
    .option("startingOffsets", "earliest") \
    .load()

24/03/21 02:38:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [13]:
from pyspark.sql import functions as F
F.current_timestamp()

Column<'current_timestamp()'>

In [3]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)
    print('inside')
    print(first_row)
    print(mini_batch)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

print(query)

24/03/20 03:09:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pr/p60v3pdj2qzdj7m2d40vmnqc0000gn/T/temporary-7852d0a8-679b-4a12-ae9d-226772b2c5b3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/20 03:09:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/20 03:09:17 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


inside
[Row(key=None, value=bytearray(b'{"pickup_datetime": "2019-10-01 00:26:02", "dropoff_datetime": "2019-10-01 00:39:58", "pu_location_id": 112, "do_location_id": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips-2', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 20, 2, 55, 11, 293000), timestampType=0)]
DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]
Row(key=None, value=bytearray(b'{"pickup_datetime": "2019-10-01 00:26:02", "dropoff_datetime": "2019-10-01 00:39:58", "pu_location_id": 112, "do_location_id": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips-2', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 20, 2, 55, 11, 293000), timestampType=0)


24/03/20 03:09:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Row(lpep_pickup_datetime=None, lpep_dropoff_datetime=None, PULocationID=None, DOLocationID=None, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [16]:
query.stop()

In [17]:
from pyspark.sql import types
from pyspark.sql import functions as F

schema = types.StructType() \
    .add("pickup_datetime", types.StringType()) \
    .add("dropoff_datetime", types.StringType()) \
    .add("pu_location_id", types.IntegerType()) \
    .add("do_location_id", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [19]:
popular_destinations = green_stream \
    .withColumn('timestamp', F.current_timestamp()) \
    .groupBy([F.window(F.col("timestamp"), "5 minutes"), "do_location_id"]) \
    .count() \
    .sort(F.col('count').desc())

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/21 02:45:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pr/p60v3pdj2qzdj7m2d40vmnqc0000gn/T/temporary-909f0770-9215-4950-89cf-47a6e80dadcc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/21 02:45:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/21 02:45:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+--------------+-----+
|window                                    |do_location_id|count|
+------------------------------------------+--------------+-----+
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|74            |17741|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|42            |15942|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|41            |14061|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|75            |12840|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|129           |11930|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|7             |11533|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|166           |10845|
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|236           |7913 |
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|223           |7542 |
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|238           |7318 |
|{2024-03-21 02:45:00, 2024-03-21 02:50:00}|8

In [ ]:
query.stop()

In [4]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/20 03:17:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pr/p60v3pdj2qzdj7m2d40vmnqc0000gn/T/temporary-ecc6bd54-8e9c-4d32-9903-72ab408f0c5e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/20 03:17:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/20 03:17:48 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(pickup_datetime='2019-10-01 00:26:02', dropoff_datetime='2019-10-01 00:39:58', pu_location_id=112, do_location_id=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


24/03/20 03:17:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [18]:
query.stop()